#### Loading libraries

In [1]:
# Load tensorflow
import tensorflow as tf
# Below command is to avoid the known bug which prevents computation on some GPU devices
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# Load preprocessing tools
from scipy.ndimage.filters import gaussian_filter
from tensorflow.keras.utils import to_categorical
from PIL import Image
# Load model building blocks
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# Load pre-trained model library
from tensorflow.keras import applications
# Load miscelaneous libraries
import numpy as np
import os
import matplotlib.pylab as plt
import gc
# Settings to set the max output of numpy to infty
# import sys
# np.set_printoptions(threshold=sys.maxsize)

#### Define paths

In [ ]:
image_path = os.path.join('/home', 'renat_sergazinov', 
                          'python-git-workspace', 'PhotoForceReconML', 'image_data', 'test')
labels_path = os.path.join('/home', 'renat_sergazinov', 
                          'python-git-workspace', 'PhotoForceReconML', 'labels', 'test')
models_path = os.path.join('/home', 'renat_sergazinov', 
                          'python-git-workspace', 'PhotoForceReconML', 'models')

#### Define image processor and data generator

In [ ]:
%run '/home/renat_sergazinov/python-git-workspace/PhotoForceReconML/data_loader.py'

#### Load the data

In [6]:
# Extract image paths
img_paths = {}
for i in range(2, 7):
    path_prefix = os.path.join(image_path, str(i))
    img_paths[i] = [os.path.join(path_prefix, name) for name in sorted(os.listdir(path_prefix), key = sorter)]

# Apply data generators
gaussblur = GaussBlur(1)
params = {'dim': (128, 128), 
          'n_channels': 3, 
          'rescale': 1 / 255, 
          'preprocessing_func': gaussblur.blur
          }
test_generator = {}
for i in range(2, 7): test_generator[i] = DataGenerator(img_paths[i], **params)
    
# Load images as matrix
X = test_generator[2].generate()
for i in range(3, 7): X = np.concatenate((X, test_generator[i].generate()), axis = 0)

In [7]:
# Load labels
y_lm = dict()
for i in range(2, 7): y_lm[i] = np.load(os.path.join(labels_path, str(i), 'mags.npy'))
y_class = np.repeat(2, y_lm[2].shape[0])
for i in range(3, 7): y_class = np.concatenate((y_class, np.repeat(i, y_lm[i].shape[0])), axis = 0)
y_class -= 2
y_class = to_categorical(y_class)

#### Load and test models

In [8]:
model_class = load_model(os.path.join(models_path, 'xception_num_forces.h5'))        

In [9]:
ev = model_class.evaluate(X, y_class, batch_size=30, verbose = 0)
print("Accuracy: ", ev[1])
print("Error: ", ev[0])

Accuracy:  0.9835000038146973
Error:  0.05906671658158302
